<a href="https://colab.research.google.com/github/dillsvarma/Azure/blob/master/expense_chart_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load CSV File from Local

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sam4.csv to sam4.csv
User uploaded file "sam4.csv" with length 7097 bytes


In [2]:
#install pyspark

!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 36.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=64bd8955135996267386efb0843f287c8e6b80db37d7fc30d14845b15af9bb48
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
# Create a Dataframe using CSV Data

from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
 
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.format("csv").option("header", "true").load("sam4.csv")
sampled_data = df.select('description','credit').sample(True, 0.8).toPandas()
 

In [4]:
# CUSTOMIZE DATA FOR GRAPH INPUT
# CATEGORIZE THE EXPENSES BASED ON THE STRING SEARCH/MATCH TECHNIQUES

import re
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
def strres(val):
  if(val):
    if(val.startswith('   BY TRANSFER')  ):
      return 'UPI/NEFT/IMPS'
    elif(val.startswith('   ATM WDL')):
      return 'ATM'
    elif(val.startswith('   by debit card')):
      return 'CARD'
    elif(val.startswith('   CREDIT INTEREST')):
      return 'INTEREST'
    elif(val.startswith('   TO TRANSFER-UPI')):
      return 'UPI'
    elif(val.startswith('   TO TRANSFER-NEFT')):
      return 'NEFT'
    elif(val.startswith('   TO TRANSFER-IMPS')):
      return 'IMPS'
    elif(val.startswith('   BULK POSTING')):
      return 'OTHER'
    elif(val.startswith('   TO TRANSFER-INB  ')):
      return 'OTHER'
new1=None
udf_func = udf(lambda x: strres(x),returnType=StringType())
#tf = df.select(*[udf_func(col(c)) for c in df.columns])
tf = df.withColumn("desc",udf_func(df.Description))

tf.show()
#t = valid_email('srinivasadileepkv.patsamatla@t-mobile.com')
#print(t)

+----------+----------+------------------+----+------+-------+--------+-------------+
|  Txn Date|Value Date|       Description| ref| Debit| Credit| Balance|         desc|
+----------+----------+------------------+----+------+-------+--------+-------------+
|2022-03-02|  2-Mar-22|TO TRANSFER-UPI/DR|null|370.77|       |41790.65|         null|
|2022-03-02|  2-Mar-22|   by debit card-O|null| 76.58|       |41714.07|         CARD|
|2022-03-02|  2-Mar-22|   BY TRANSFER-UPI|null|  null|  36000|77714.07|UPI/NEFT/IMPS|
|2022-03-05|  5-Mar-22|   BULK POSTING-UP|null|      | 111000|78714.07|        OTHER|
|2022-03-05|  5-Mar-22|   TO TRANSFER-UPI|null|    99|       |78615.07|          UPI|
|2022-03-05|  5-Mar-22|   TO TRANSFER-UPI|null|    99|       |78516.07|          UPI|
|2022-03-05|  5-Mar-22|   TO TRANSFER-UPI|null|   300|       |78216.07|          UPI|
|2022-03-05|  5-Mar-22|   BULK POSTING-AC|null|      |  11200|78234.07|        OTHER|
|2022-03-05|  5-Mar-22|   TO TRANSFER-UPI|null|   290|

In [5]:
# DOWNLOAD TRANSFORMED DATA IF NEEDED FROM GOOGLE COLAB

from google.colab import files

tf.toPandas().to_csv('df.csv')
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# CREATE GRAPH INPUT DATA FOR DEBIT TRANSACTIONS


from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import month,to_date, unix_timestamp
from datetime import datetime
import calendar

gf=tf.filter(tf.Credit == ' ').groupBy('desc',month(to_date(unix_timestamp(col('Txn Date'), 'yyyy-MM-dd').cast("timestamp")).alias('month'))).agg(_sum(tf['Debit'].cast(DecimalType())).alias('sum')).na.fill(value=0).na.fill("NA")

gf1=gf.select('desc',col("month(to_date(CAST(unix_timestamp(Txn Date, yyyy-MM-dd) AS TIMESTAMP)) AS month)").alias("month"),"sum")



In [7]:
# fOR GETTING MONTH 
gf1.createOrReplaceTempView("gf1")

gf2 = spark.sql(
    "select desc, sum, month from gf1") \
    .toPandas()

d = dict(enumerate(calendar.month_abbr))
gf2['Month'] = gf2['month'].map(d)

print(gf2)

   desc    sum  month Month
0   UPI  18964      3   Mar
1  CARD    602      3   Mar
2    NA    371      3   Mar
3   ATM  31000      4   Apr
4    NA   1220      4   Apr
5   UPI  25252      4   Apr


In [8]:
# plot the graph using plotly

import plotly.express as px
import plotly.graph_objects as go

#x1 = gf.select("desc").rdd.flatMap(lambda f: f).collect()
#y1 = gf.select("sum").rdd.flatMap(lambda g: g).collect()

#data = gf2.query("")

#fig = px.bar(gf2, x='desc', y='sum',
#             hover_data=['sum', 'desc'], color='month',
#             labels={'desc':'TransactionType'}, height=400)
#fig = go.Figure(data=[px.histogram(gf2, x="desc", y="sum",
#             color='Month', barmode='group',
#             height=400)])

fig = px.bar(gf2, y='sum', x='desc', text=gf2['sum'].apply(lambda x: '{:,}'.format(x)), color='Month')

layout = go.Layout(yaxis=dict(tickformat=".2f"))
fig = go.Figure(data=fig, layout=layout)

fig.update_layout(title_text='Debit Transactions Mar-Apr-2022', yaxis_tickformat = ',')
fig.update_traces(width=0.3)

fig.show()

## for credit transactions

In [9]:
#####


from pyspark.sql.functions import sum as _sum
import datetime
from pyspark.sql import functions as F

cf=tf.filter(tf.Debit == ' ').groupBy('desc',month(to_date(unix_timestamp(col('Txn Date'), 'yyyy-MM-dd').cast("timestamp")).alias('month'))).agg(_sum(tf['Credit'].cast(DecimalType())).alias('sum')).na.fill(value=0).na.fill("NA")
cf1=cf.select('desc',col("month(to_date(CAST(unix_timestamp(Txn Date, yyyy-MM-dd) AS TIMESTAMP)) AS month)").alias("month"),"sum")



In [10]:
#tf1 = tf.withColumn('Txn Date1', 
#                   to_date(unix_timestamp(col('Txn Date'), 'MM/dd/yy').cast("timestamp")))
#tf1.select(month('Txn Date1')).show()
#tf1.printSchema()

In [11]:
cf1.show()

+-------------+-----+------+
|         desc|month|   sum|
+-------------+-----+------+
|        OTHER|    3|122200|
|UPI/NEFT/IMPS|    3|195608|
|     INTEREST|    3| 11325|
|UPI/NEFT/IMPS|    4| 93168|
+-------------+-----+------+



In [12]:
cf1.createOrReplaceTempView("cf1")

cf2 = spark.sql(
    "select desc, sum, month from cf1") \
    .toPandas()

d = dict(enumerate(calendar.month_abbr))
cf2['Month'] = cf2['month'].map(d)

print(cf2)


            desc     sum  month Month
0          OTHER  122200      3   Mar
1  UPI/NEFT/IMPS  195608      3   Mar
2       INTEREST   11325      3   Mar
3  UPI/NEFT/IMPS   93168      4   Apr


In [14]:
import plotly.express as px


fig = px.bar(cf2, y='sum', x='desc', text=cf2['sum'].apply(lambda x: '{:,}'.format(x)), color='Month')


fig.update_layout(title_text='Credit Transactions Mar-Apr-2022', yaxis_tickformat = ',')
fig.update_traces(width=0.3)